In [7]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OrdinalEncoder,FunctionTransformer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
import joblib
from sklearn.preprocessing import OneHotEncoder


In [8]:
full_data=pd.read_csv("/Users/joaoalmeida/Desktop/test.csv",index_col=[0])

In [9]:
ord_cols=["BISHOP_SCORE",
"BISHOP_CONSISTENCIA",
"BISHOP_DESCIDA",
"BISHOP_DILATACAO",
"BISHOP_EXTINCAO",
"BISHOP_POSICAO"]

int_cols = ["A_PARA","A_GESTA","EUTOCITO_ANTERIOR","FORCEPS_ANTERIOR","CESARIANAS_ANTERIOR",
    "IDADE_MATERNA",
    "PESO_INICIAL",
    "IMC",
    "NUMERO_CONSULTAS_PRE_NATAL",
    "IDADE_GESTACIONAL_ADMISSAO",
    "SEMANAS_GESTACAO_PARTO",
    "PESO_ADMISSAO_INTERNAMENTO",
    "ESTIMATIVA_PESO_ECO_30",
    "ESTIMATIVA_PESO_ECO_31",
    "ESTIMATIVA_PESO_ECO_32",
    "ESTIMATIVA_PESO_ECO_24",
    "VENTOSAS_ANTERIOR",
"ESTIMATIVA_PESO_ECO_25",
"ESTIMATIVA_PESO_ECO_26",
"ESTIMATIVA_PESO_ECO_27",
"ESTIMATIVA_PESO_ECO_28",
"ESTIMATIVA_PESO_ECO_29",
"ESTIMATIVA_PESO_ECO_33",
"ESTIMATIVA_PESO_ECO_34",
"ESTIMATIVA_PESO_ECO_35",
"ESTIMATIVA_PESO_ECO_36",
"ESTIMATIVA_PESO_ECO_37",
"ESTIMATIVA_PESO_ECO_38",
"ESTIMATIVA_PESO_ECO_39",
"ESTIMATIVA_PESO_ECO_40",
"ESTIMATIVA_PESO_ECO_41"]
cat_cols = list(set(full_data.columns)  - set(int_cols) -set(ord_cols))

In [10]:
ord_cols=[col for col in ord_cols if col  in full_data.columns ]
int_cols=[col for col in int_cols if col  in full_data.columns ]

In [11]:
full_data[cat_cols]=full_data[cat_cols].astype(str)

In [30]:
#list(full_data["G_TERAPEUTICA"].unique())
#transform all from SIM/NAO to S/N e similar
#BACIA [nan 'ADEQUADA' 'LIMITE' 'INADEQUADA' 'A' 'L']
standard_map={-1:np.nan,-7:np.nan,"ADEQUADA":"A","LIMITE":"L","DESCONHECIDO,RH_DESCONHECIDO":np.nan,"DESCONHECIDO,":np.nan,"SIM":"S","NAO":"N","-1":np.nan,-1:np.nan,"NS":np.nan,",":np.nan,"Sim":"S","UNKNOWN":np.nan,"Desconhecida":np.nan,"Desconhecido":np.nan,"DESCONHECIDO":np.nan," ":np.nan,"Desconhecido,":np.nan,"  ":np.nan}
def standardize_null(x,mapping):
    if x in mapping.keys():
        return mapping[x]
    if pd.isna(x):
        return np.nan
    return x

for col in full_data.columns:
    full_data[col]=full_data[col].apply(standardize_null,mapping=standard_map)

In [31]:
numeric_transformer = Pipeline(
     steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        )
    ]
)
ordinal_transformer= Pipeline(
    steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        )#,
    ]
)

categorical_transformer = Pipeline(
    steps=[ #("fun_str", fun_str),
       
        (
            "imputer",
            SimpleImputer(
                missing_values="nan", strategy="constant", fill_value="Unknown"
            )
        ),

                ("OneHotEncoder", OneHotEncoder(sparse_output=False))

    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, int_cols),
        ("ord",ordinal_transformer,ord_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)
outlier_pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

In [32]:
X_treated_outlier=outlier_pipeline.fit_transform(full_data)

In [43]:
joblib.dump(outlier_pipeline, '../outlier_pipeline.sav')

['../outlier_pipeline.sav']

In [33]:
X_treated_df=pd.DataFrame(X_treated_outlier,columns=outlier_pipeline.get_feature_names_out())
X_treated_df.columns=[l.replace("num__","").replace("cat__","") for l in X_treated_df.columns]

In [44]:
full_data.columns

Index(['IDADE_MATERNA', 'GS', 'PESO_INICIAL', 'IMC', 'A_PARA', 'A_GESTA',
       'EUTOCITO_ANTERIOR', 'VENTOSAS_ANTERIOR', 'CESARIANAS_ANTERIOR',
       'TIPO_GRAVIDEZ', 'VIGIADA', 'NUMERO_CONSULTAS_PRE_NATAL',
       'VIGIADA_PARICULAR', 'VIGIADA_CENTRO_SAUDE', 'VIGIADA_NESTE_HOSPITAL',
       'APRESENTACAO_ADMISSAO', 'IDADE_GESTACIONAL_ADMISSAO',
       'TRAB_PARTO_ENTRADA_ESPONTANEO', 'TIPO_PARTO', 'APRESENTACAO_NO_PARTO',
       'TRAB_PARTO_NO_PARTO', 'SEMANAS_GESTACAO_PARTO', 'GRUPO_ROBSON',
       'silo'],
      dtype='object')

In [34]:

# identify outliers in the training dataset
lof = LocalOutlierFactor(novelty=True)
yhat = lof.fit(X_treated_df)

In [35]:
yhat = lof.predict(X_treated_df)



/opt/homebrew/Caskroom/miniforge/base/envs/dq/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


In [36]:
# select all rows that are not outliers
mask = yhat != -1
gg=X_treated_df.loc[mask, :]
# summarize the shape of the updated training dataset

In [37]:
nmask = yhat != 1
outs=full_data.loc[nmask, :]
# summarize the shape of the updated training dataset


In [38]:
outs

,IDADE_MATERNA,GS,PESO_INICIAL,IMC,A_PARA,A_GESTA,EUTOCITO_ANTERIOR,VENTOSAS_ANTERIOR,CESARIANAS_ANTERIOR,TIPO_GRAVIDEZ,...,VIGIADA_NESTE_HOSPITAL,APRESENTACAO_ADMISSAO,IDADE_GESTACIONAL_ADMISSAO,TRAB_PARTO_ENTRADA_ESPONTANEO,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON,silo
236,29.0,"O,RH_POSITIVO",80.0,31.2,6.0,9.0,4.0,NaN,2.0,nan,...,S,apr.cefala.3,33.0,nan,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,33.0,10.0,1
277,32.0,"A,RH_POSITIVO",62.0,23.3,0.0,1.0,NaN,NaN,NaN,ESPONTANEA,...,S,nan,25.0,nan,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,30.0,10.0,1
821,30.0,"A,RH_POSITIVO",82.0,NaN,4.0,8.0,NaN,NaN,4.0,ESPONTANEA,...,S,nan,39.0,nan,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,39.0,5.0,1
1396,19.0,"O,RH_POSITIVO",37.0,NaN,NaN,1.0,NaN,NaN,NaN,ESPONTANEA,...,S,nan,41.0,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,41.0,1.0,1
1737,39.0,"O,RH_POSITIVO",NaN,NaN,NaN,1.0,NaN,NaN,NaN,ESPONTANEA,...,nan,nan,NaN,nan,Grande extração pélvica,Pélvica,Espontâneo,22.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68333,15.0,"A,RH_POSITIVO",87.0,36.7,NaN,1.0,NaN,NaN,NaN,ESPONTANEA,...,S,nan,36.1,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,36.1,10.0,9
70148,30.0,"A,RH_POSITIVO",101.0,NaN,1.0,2.0,1.0,NaN,NaN,ESPONTANEA,...,S,apr.cefala.3,40.6,nan,Parto eutócico cefálico,Cefálica de vértice,Indução com dinoprostona,41.0,4.0,9
70174,24.0,"O,RH_POSITIVO",62.0,46.1,NaN,1.0,NaN,NaN,NaN,ESPONTANEA,...,nan,nan,39.0,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,39.0,1.0,9
71520,25.0,"B,RH_POSITIVO",96.0,NaN,3.0,4.0,1.0,NaN,2.0,ESPONTANEA,...,S,apr.cefala.3,39.0,nan,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,39.1,5.0,9


In [39]:

# identify outliers in the training dataset
ee = EllipticEnvelope(contamination=0.01)
yhat = ee.fit_predict(X_treated_df )

/opt/homebrew/Caskroom/miniforge/base/envs/dq/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


In [40]:
mask = yhat != -1
gg=X_treated_df.loc[mask, :]
# summarize the shape of the updated training dataset


In [41]:
joblib.dump(ee, '../EllipticEnvelope.sav')

['../EllipticEnvelope.sav']

In [42]:
joblib.dump(lof, '../LocalOutlierFactor.sav')

['../LocalOutlierFactor.sav']